# 所需环境

transformers版本号为2.1.1，pytorch为1.2.0。

In [48]:
import transformers
print(transformers.__version__)

I0415 15:24:11.910086 140436257081152 file_utils.py:39] PyTorch version 1.2.0 available.


2.1.1


在requirments.txt中，列出了所需要的其他的包，如下所示。

In [49]:
!cat requirements.txt

transformers==2.1.1
torch
numpy
tqdm
sklearn
keras
tb-nightly
future
thulac


# 目录结构以及重要文件

当前文件夹中包含的文件如下。其中**doupo**文件夹下包含“斗破苍穹”的示例任务，用以简单说明如何构建字典、tokenization、以及在一个小文件上从头训练一个指定层数的GPT模型；**pretrain**文件夹下包含预训练任务，主要用以说明如何在260万篇新闻文档上预训练一个12层的GPT2；**dataaugmentation**文件夹下包含数据增强任务，主要用以说明如何利用预训练的GPT2模型在较小的文档集上进行微调，并用于数据增强。

In [50]:
%%bash
tree -L 1 .

.
├── cache
├── config
├── dataaugmentation
├── eval.py
├── generate.py
├── generate_texts.py
├── LICENSE
├── pretrain
├── README.md
├── requirements.txt
├── sample
├── tasks
├── tokenizations
├── Train_GPT2_from_scratch.ipynb
├── train.json
├── train_on_small_file.py
└── train_single.py

7 directories, 10 files


需要重点加以说明的是，上述列表中的tokenizations文件夹，train_single.py, 和train_on_small_file.py。

## Tokenization

`tokenizations`提供了各种tokenization的方法，具体到本代码库中，使用的是该文件夹中的`tokenization_chars.py`文件。

tokenization的第一阶段是将纯文本文件依照字典文件vocab.txt进行划分为token，第二阶段是将token转化为字典文件中token对应的数字。举例来说，"[CLS]《斗破苍穹》天蚕土豆[SEP][CLS] ..."在第一阶段被分割为列表：['[CLS]', '《', '斗', '破', '苍', '穹', '》', '天', '蚕', '土', '豆', '[SEP]', '[CLS]', ...]，其中[CLS], [SEP]在vocab.txt对应一个单独的占位符；第二阶段在查找vocab.txt之后，将字符串列表转化为整型数列表：101 517 3159 4788 5721 4957 518 1921 6014 1759 6486 102 101。

如下是tokenization第一阶段的代码，输入为text字符串，输出为字符串列表result，对于形如[.\*]的特殊字符使用栈来处理：如果该特殊字符出现在vocab.txt中，则被提取为一个单独的字符串存入列表中，如161行所示；反之若没有出现在vocab.txt中，则逐字符加入到列表中，如163-165行所示。

In [51]:
!perl -ne 'print "$. $_" if ($.>=140 and $.<=176)' tokenizations/tokenization_chars.py

140     def _tokenize(self, text):
141         #pdb.set_trace()
142         stack = []
143 
144         result = []
145         i = 0
146         len_txt = len(text)
147         while(i<len_txt):
148             char = text[i]
149             if(len(stack)==0):
150                 if(char != '['):
151                     result.append(char)
152                 else:
153                     stack.append('[')
154             else:
155                 if(char == ']'):
156                     stack.append(char) # don't forget to append it firstly
157                     # process the content in the stack
158                     seq = "".join(stack)
159                     if(seq in self.vocab):
160                         # the [.*] stuff appeared in the vocabulary, e.g. [UNK], [CLS], [SEP], ...
161                         result.append(seq)
162                     else:
163                         # other [.*] stuff which are not valid element in the vocabulary
164                        

tokenization的第二部分如下所示，对第一阶段获得的result列表中的token字符串进行查表操作，如果没有出现在vocab.txt中，那么就以[UNK]对应的id来代替。

In [52]:
!perl -ne 'print "$. $_" if ($.>=178 and $.<=182)' tokenizations/tokenization_chars.py

178     def _convert_token_to_id(self, token):
179         """ Converts a token (str/unicode) in an id using the vocab. """
180         #if(token not in self.vocab):
181         #    print(token)
182         return self.vocab.get(token, self.vocab.get(self.unk_token))


注意，本代码库中的tokenization方法主要为中文设计，如果训练数据中混杂有英文字符，那么就将英文单词按照char level进行划分，例如"word"会被"w","o","r","d"四个字符来代替。不同于BPE的方法，我们认为这样处理是最节省计算资源的，比起BPE能够更从容应对几十GB的中文训练语料。从具体的实际效果来看，在斗破苍穹的语料上进行tokenization，普通方法需要耗时77.9秒，使用了我们的方法之后，耗时12.1秒，用时为原来的15%。

## 两种训练方法

`train_single.py`：当输入的训练数据为一篇完整的长文档（例如小说）或是一个单一的大型文档，此时使用`train_single.py`。有两方面的原因。（1）train_single.py在训练过程中不对training samples进行random shuffle，保持training samples之间的顺序，因此完整的长文档使用train_single.py。（2）对顺序无关的超多的样本，例如包含260万篇新闻的大型训练集，在训练过程中进行random shuffle的代价是巨大的，包括shuffle的代价，重新划分training sample的代价和tokenization的代价，并且在大型训练集上我们进行预训练的轮数有限，1到2轮就能获得一个较好的预训练模型。综合考虑shuffle的代价和收益，因此也将大型训练集中的多个样本拼接为一个单一的大型文档，使用train_single.py。我们在《从头训练一个GPT2模型》一节对此进行更详细的说明。

`train_on_small_file.py`：当输入的训练数据为多个样本，并且这多个样本的总体积较小时，将这些样本汇总到一个文件，每个样本占据一行，然后使用`train_on_small_file.py`。有如下两方面原因：（1）train_on_small_file.py针对每个样本建立一个单独的training sample送入Transformer Encoder中，如果一个样本的长度不足Transformer Encoder的长度（在本代码库中用n_ctx表示），那么不足部分以\[PAD\]来补足；超出n_ctx的部分直接截断丢弃不用。这一点和train_single.py不同，train_single.py中多个较短的样本可能会占据一个Transformer Encoder的输入长度（n_ctx）。（2）train_on_small_file.py会在不同的训练轮次中，对training samples进行random shuffle，以提高训练质量。由于train_on_small_file.py的训练数据体积较小，样本数也较小，因此每轮进行random shuffle是完全可行的。所以，如果是唐诗、宋词、现代诗、意图增强等文本数据，由于其规模较小，完全可以使用`train_on_small_file.py`来完成训练或者微调，这样得到的模型质量要高于`train_single.py`训练得到的模型。我们在《微调GPT2模型进行数据增强》一节对此进行更详细的说明。

# 从头训练一个GPT2模型

## 训练数据

此处我们使用一个规模较小的文件用以验证从头训练一个GPT2模型的可行性。该文件为斗破苍穹小说文本文件，规模为16MB，16万行。如下是该文件的一些基本信息。

In [54]:
!cd tasks/doupo; ls -laht rawdata/train_raw.txt; wc -l rawdata/train_raw.txt; head -n 15 rawdata/train_raw.txt; echo "..."; cd ../..;

-rw-rw-r-- 1 weijing weijing 16M Apr  5 10:54 rawdata/train_raw.txt
162111 rawdata/train_raw.txt
《斗破苍穹》天蚕土豆

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何关系。
在线阅读：http://www.shuyaya.com/read/18/
--------------------------------------------------

第一章 陨落的天才

    “斗之力，三段！”

    望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛…

    “萧炎，斗之力，三段！级别：低级！”测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来…

    中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。
...


如果没有rawdata/train_raw.txt文件，那么运行`!cd tasks/doupo; bash train.sh`，该脚本帮助建立对应文件夹并下载train_raw.txt。如下是train.sh的部分信息。首先创建raw_data文件夹，divide文件夹，tokenized文件夹，model文件夹和config文件夹。其中，raw_data文件夹用于存储原始的训练数据和进行基本预处理之后的训练数据；divide文件夹用于存储分割后的大文件（在train_single.py中有体现）；tokenized文件夹用于存储token转为数字id之后的文件；model文件夹用于储存训练好的模型；config文件夹用于储存模型的基本配置信息。

`tasks/doupo/train.sh`的第26行用于从公网下载原始训练数据并存储到rawdata文件夹下的train_raw.txt。

In [55]:
!cd tasks/doupo; perl -ne 'print "$. $_" if ($.>=1 and $.<=29)' train.sh; cd ../..;

1 job_dir="tasks/doupo"
2 
3 cd ../..
4 
5 if [ ! -e $job_dir/rawdata ]; then
6     mkdir $job_dir/rawdata
7 fi
8 
9 if [ ! -e $job_dir/divide ]; then
10     mkdir $job_dir/divide
11 fi
12 
13 if [ ! -e $job_dir/tokenized ]; then
14     mkdir $job_dir/tokenized
15 fi
16 
17 if [ ! -e $job_dir/model ]; then
18     mkdir $job_dir/model
19 fi
20 
21 if [ ! -e $job_dir/config ]; then
22     mkdir $job_dir/config
23 fi
24 
25 if [ ! -e $job_dir/rawdata/train.txt ]; then
26     wget -c -O $job_dir/rawdata/train_raw.txt https://github.com/GaoPeng97/transformer-xl-chinese/blob/master/data/doupo/train.txt?raw=true
27     # remove empty lines
28     python $job_dir/format_raw_txt.py $job_dir/rawdata/train_raw.txt $job_dir/rawdata/train.txt 
29 fi


`tasks/doupo/train.sh`的第28行调用`format_raw_txt.py`，用于移除原始文件中的空行和去除一行中左侧多余的空格(lstrip)。

In [56]:
!cd tasks/doupo; cat -n format_raw_txt.py; cd ../..;

     1	'''
     2	This script is used to remove empty lines in the input file 
     3	'''
     4	import sys
     5	
     6	def format(in_filename, out_filename):
     7	    with open(out_filename, "w") as fOut:
     8	        with open(in_filename, "r") as fIn:
     9	            for line in fIn:
    10	                if(line!=""):
    11	                    fOut.write(line.lstrip())
    12	
    13	
    14	def test():
    15	    in_filename="./rawdata/train_raw.txt"
    16	    out_filename="./rawdata/train.txt"
    17	    format(in_filename, out_filename)
    18	
    19	
    20	if __name__=="__main__":
    21	    if(len(sys.argv)<3):
    22	        print("usage: python format_raw_txt.py in_filename out_filename")
    23	    else:
    24	        in_filename = sys.argv[1]
    25	        out_filename = sys.argv[2]
    26	        format(in_filename, out_filename)


## 建立vocab.txt

当前多个模型都直接使用了BERT chinese字表，该字表拥有字符21128个。但是该字表有如下的问题：不包含大写字母A到Z，不包含制表符，空格等字符。我们可以将这些字符追加到BERT chinese字表之后，如`train.sh`代码中的第31行到44行所示，将新的字符表保存在doupo/config/vocab.txt文件中。

In [57]:
!cd tasks/doupo; perl -ne 'print "$. $_" if ($.>=31 and $.<=44)' train.sh; cd ../..;

31 vocab_size=21128
32 declare -a additional_chars=("“" "”" "…" "’" "‘" "—" " " "\t" "\`")
33 new_vocab_size=$(($vocab_size+${#additional_chars[@]}+26))
34 echo 'setting config/vocab.txt and config/model_config.json'
35 if [ ! -e $job_dir/config/vocab.txt ]; then
36     wget -c -O $job_dir/config/vocab.txt https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt
37     # append new characters to the vocabulary
38     for letter in "${additional_chars[@]}"; do
39         echo -e "$letter" >> $job_dir/config/vocab.txt
40     done
41     for letter in {A..Z} ; do
42         echo $letter >> $job_dir/config/vocab.txt
43     done
44 fi


## 模型参数设置

我们可以进一步的设置模型参数，并将模型参数保存于config/model_config.json。如果不存在doupo/config/model_config.json，那么doupo/train.sh的第49行拷贝config/model_config.json模板到指定位置，并进行编辑。如下的代码第52，54，56，57行用于编辑模型参数，例如将层数修改为10层，将n_ctx和n_positions从1024修改为512。因为n_ctx是Transformer Encoder能够容纳的最大序列长度，因此减半之后可以大幅度节省训练时占用的显存，同时也考虑到512是一个合理的较长的长度，能够较大限度的捕获文本上远距离的依赖关系。此外我们将stride设置为256，也就是在一个长序列上窗口大小为512，每次移动窗口的幅度为256。

In [58]:
!cd tasks/doupo; perl -ne 'print "$. $_" if ($.>=46 and $.<=58)' train.sh; cd ../..;

46 stride=256
47 n_layers=10
48 n_ctx=512
49 if [ ! -e $job_dir/config/model_config.json ]; then
50     cp config/model_config.json $job_dir/config/model_config.json
51     # change vocabulary size
52     perl -pi -e 's/'$vocab_size'/'$new_vocab_size'/g' $job_dir/config/model_config.json
53     # change the number of layers from 12 to $n_layers
54     perl -pi -e 's/"n_layer": 12/"n_layer": '$n_layers'/g' $job_dir/config/model_config.json
55     # change the model input length from 1024 to $n_ctx
56     perl -pi -e 's/"n_ctx": 1024/"n_ctx": '$n_ctx'/g' $job_dir/config/model_config.json
57     perl -pi -e 's/"n_positions": 1024/"n_positions": '$n_ctx'/g' $job_dir/config/model_config.json
58 fi


我们可以在`config/model_config.json`中查看即将要训练的模型的参数。

In [59]:
!cd tasks/doupo; cat -n config/model_config.json; cd ../..;

     1	{
     2	  "initializer_range": 0.02,
     3	  "layer_norm_epsilon": 1e-05,
     4	  "n_ctx": 512,
     5	  "n_embd": 768,
     6	  "n_head": 12,
     7	  "n_layer": 10,
     8	  "n_positions": 512,
     9	  "vocab_size": 21163
    10	}

## 数据预处理

必要的数据预处理发生于`train_single.py`的`build_files`文件中，如下所示。`build_files`包含有多个参数，其中有必要说明的是`num_pieces`和`full_tokenizer`。`full_tokenizer`调用的是《Tokenization》一小节中提到的两阶段方法，效率较高。`num_pieces`将一个大文件分割为若干个小文件，以减小训练时IO一个大文件带来的内存压力。
如下，第26行到第35行用以分割大文件到divide文件夹。分割之后，每一行补充[CLS]字符，回车符'\\n'转为[SEP]字符，然后调用`full_tokenizer`将token转为id。

举例来说，"《斗破苍穹》天蚕土豆"这一行转为"[CLS]《斗破苍穹》天蚕土豆[SEP]"并转为对应的数字id。

In [60]:
!perl -ne 'print "$. $_" if ($.>=18 and $.<=62)' train_single.py

18 def build_files(raw_data_path, divide_path, tokenized_data_path, full_tokenizer, num_pieces):
19     if not os.path.exists(tokenized_data_path):
20         os.mkdir(tokenized_data_path)
21     if not os.path.exists(divide_path):
22         os.mkdir(divide_path)
23     print("now time: ", datetime.now())
24     print("begin to divide raw text ...")
25 
26     writers = [open(divide_path + 'divide_piece_{}.txt'.format(i), 'w') for i in range(0,num_pieces)]
27 
28     with open(raw_data_path, 'r', encoding='utf8') as f:
29         line_num = 0
30         for line in f:
31             writers[line_num % num_pieces].write("%s" % line)
32             line_num += 1
33     
34     for i in range(0, num_pieces):
35         writers[i].close()
36     
37     print('now time: ', datetime.now())
38     print("begin making tokenization ...")
39     files = [filename for filename in os.listdir(divide_path) if f!='.gitignore']
40     for i, filename in enumerate(files):
41         if(os.path.isdir(

## 确定训练过程中的其他参数

在确定模型参数之后，再定义训练过程中的其他参数，如下为`tasks/doupo/train.sh`中该部分代码，这些参数包括：迭代的轮数`epochs`，训练时的`batch_size`，每隔多少轮输出一次NLL的`log_step`，以及使用哪些显卡的`device`。考虑到具体的硬件资源，此处使用两块显卡——0号和1号，以及设置batch size为32。

In [78]:
!cd tasks/doupo; perl -ne 'print "$. $_" if ($.>=60 and $.<=103)' train.sh; cd ../..;

60 raw_data_path=$job_dir/rawdata/train.txt
61 tokenizer_path=$job_dir/config/vocab.txt
62 tokenized_data_path=$job_dir/tokenized/
63 divide_path=$job_dir/divide/
64 model_config=$job_dir/config/model_config.json
65 epochs=30
66 batch_size=32
67 log_step=100
68 output_dir=$job_dir/model/
69 num_pieces=1
70 
71 if [ ! -e $job_dir/tokenized/tokenized_train_0.txt ]; then
72     # tokenization then run the training
73     python train_single.py \
74         --raw_data_path $raw_data_path \
75         --tokenizer_path $tokenizer_path \
76         --tokenized_data_path $tokenized_data_path \
77         --divide_path $divide_path \
78         --model_config $model_config \
79         --epochs $epochs \
80         --batch_size $batch_size \
81         --stride $stride \
82         --log_step $log_step \
83         --output_dir $output_dir \
84         --num_pieces $num_pieces \
85         --raw \
86         --ignore_intermediate_epoch_model
87 else
88     # run the training on the tokenized fi

## 进行训练

确定完毕模型参数和训练过程中的其他参数之后，可以进入到tasks/doupo文件夹，然后运行train.sh。

In [62]:
!cd tasks/doupo; bash train.sh; cd ../..;

setting config/vocab.txt and config/model_config.json
I0416 01:14:42.333565 140639139563328 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=32, device='0,1', divide_path='tasks/doupo/divide/', epochs=30, fp16=False, fp16_opt_level='O1', gradient_accumulation=1, ignore_intermediate_epoch_model=True, log_step=100, lr=0.00015, max_grad_norm=1.0, model_config='tasks/doupo/config/model_config.json', num_pieces=1, output_dir='tasks/doupo/model/', pretrained_model='', raw=False, raw_data_path='tasks/doupo/rawdata/train.txt', segment=False, stride=256, tokenized_data_path='tasks/doupo/tokenized/', tokenizer_path='tasks/doupo/config/vocab.txt', warmup_steps=2000)
config:
{
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false

## 保存模型

模型保存于`tasks/doupo/model/final_model`中，该文件夹下有两个文件，`config.json`，`pytorch_model.bin`。在`config.json`中包含了模型的基本参数。

In [71]:
!ls -laht tasks/doupo/model/final_model

total 344M
-rw-rw-r--  1 weijing weijing 344M Apr 16 08:51 pytorch_model.bin
-rw-rw-r--  1 weijing weijing  596 Apr 16 08:51 config.json
drwxrwxr-x  2 weijing weijing 4.0K Mar 31 06:32 .
drwxrwxr-x 33 weijing weijing 4.0K Mar 31 06:32 ..


`pytorch_model.bin`可以被直接加载，并能够输出模型结构。

In [76]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("tasks/doupo/model/final_model")
print(model)

I0416 10:41:52.995795 140436257081152 configuration_utils.py:148] loading configuration file tasks/doupo/model/final_model/config.json
I0416 10:41:52.998218 140436257081152 configuration_utils.py:168] Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 21163
}

I0416 10:41:52.999756 140436257081152 modeling_utils.py:334] loading weights file tasks/doupo/model/final_model/pytorch_model.bin


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21163, 768)
    (wpe): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

若要查看模型中的参数个数，可以使用如下命令。

In [81]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("The number of parameters in the model is %s" % pytorch_total_params)

The number of parameters in the model is 87526656


## 在训练好的模型上做Inference，生成文本

`generate.py`脚本使用top-k sampling生成文本。`tasks/doupo/generate.sh`定义了top-k sampling的参数，如下所示，topk=50，nsamples=10，temperature=0.8，length=18，prefix=[SEP][CLS]萧炎。topk越小，多样性越低。temperature $T$也定义了多样性：取到词表中第i个词的概率由logits向量$z_{1:V}$和温度T决定, $q_i = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}$。具体的，T越大，向量$z_{1:V}$中各分量的绝对值就越趋向于0，造成多样性提高而采样质量下降；T越小，则各分量就越远离0，多样性下降而质量提升。一般的，top-k sampling中使用的T为0.8。

In [82]:
!cat -n tasks/doupo/generate.sh

     1	job_dir="tasks/doupo"
     2	
     3	cd ../..
     4	
     5	if [ ! -e $job_dir/outputs ]; then
     6	    mkdir $job_dir/outputs
     7	fi
     8	
     9	python generate.py \
    10	    --device 0 \
    11	    --model_path $job_dir/model/final_model/ \
    12	    --model_config $job_dir/model/final_model/model_config.json \
    13	    --tokenizer_path $job_dir/config/vocab.txt \
    14	    --temperature 0.8 \
    15	    --prefix [SEP][CLS]萧炎 \
    16	    --length 100 \
    17	    --topk 50 \
    18	    --nsamples 10 \
    19	    --save_samples \
    20	    --save_samples_path $job_dir/outputs/


In [68]:
!cd tasks/doupo;bash generate.sh;cd ../..;

I0416 09:33:09.533224 139695085782848 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=1, device='0', fast_pattern=False, length=100, model_config='tasks/doupo/model/final_model/model_config.json', model_path='tasks/doupo/model/final_model/', no_wordpiece=False, nsamples=10, prefix='[SEP][CLS]萧炎', repetition_penalty=1.0, save_samples=True, save_samples_path='tasks/doupo/outputs/', segment=False, temperature=0.8, tokenizer_path='tasks/doupo/config/vocab.txt', topk=50, topp=0)
I0416 09:33:09.818720 139695085782848 configuration_utils.py:148] loading configuration file tasks/doupo/model/final_model/config.json
I0416 09:33:09.819159 139695085782848 configuration_utils.py:168] Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output

上述文本生成结果保存于`tasks/doupo/generate.sh`中定义的`$save_samples_path`文件夹下。

# 微调GPT2模型进行数据增强

In [ ]:
!ls